In [1]:
rules = """---------
BlackJack Game Rules :
    
    Black jack is a card game played between the player and the dealer.

    At the start, both you and the dealer are dealt two cards. The dealer shows
you his second card.

    Your cards, or hand, have a score. You get your score by
adding up the values of each card in your hand. Every number card's value is
its number, and any face card is worth 10. The winner of the game is whoever's score is
closest to 21. Ties go to the dealer.

    With your two cards, you have the option to hit or stay. Hit means draw one
more card. Stay means you're happy with your score and would like to challange the dealer's hand
---------"""

In [2]:
import random



class Deck:
    
    
    def __init__(self):
        ### setting up attributes for deck class ###
        self.cards = list(['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'] * 4)
        self.values = {'A':11, 'J':10, 'Q':10, 'K':10, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10}
        self.shuffle()
        

    def shuffle(self):
        ### shuffles the deck ###
        random.shuffle(self.cards)

        
    def deal_hand(self):
        ### deals 2 cards from the deck ###
        return [self.cards.pop() for i in range(2)]
        

        
class Hand:
    
    
    def __init__(self,deck,dealer=False):
        ### setting up attributes for hand class ###
        self.deck = deck
        self.hand = []
        self.dealer = dealer
        self.value = 0  
        
        
    def __str__(self):
        hand_cards = [i for i in self.hand]
        print(f'\nDealer\'s cards : {hand_cards}')
    

    def calculate_value(self):
        ### calc hand value ###
        self.value = 0
        for card in self.hand:
            self.value += self.deck.values[card]
            if 'A' in self.hand and self.value > 21:
                ### checks for Ace in the hand ###
                self.value -= 10
        return self.value
        
    
    def draw_card(self):
        ### draw a card from the deck ###
        [self.hand.append(self.deck.cards.pop()) for i in range(1)]
        return self.hand
    
    
    def display(self):
        ### display the hand ###
        if self.dealer:
            print(f"\nFirst card in the dealer's hand is hidden : X & {self.hand[1]}")
        else:
            print(f'\nYour cards are : {self.hand} with value of : {self.calculate_value()}')

    
    
class Player:
    
              
    def __init__(self, deck):
        ### connecting attributes for the player ###
#         self.bet = 0
        self.deck = deck
        self.player_hand = Hand(deck)
        
     
    
class Chips:
    
    
    def __init__(self):
        ### setting up attributes for the chips and bets ###
        self.total = 1000
        self.bet = 0
        
    def win_bet(self):
        ### winning a bet > * 2 ###
        self.total += self.bet * 2
        return self.total
    
    def lose_bet(self):
        ### losing a bet ###
        self.total -= self.bet
        return self.total
    
    def take_bet(self):
        ### taking int as an input and checks vs chip pool ###
        ## try & except ##
        try:
            self.bet = int(input('\nHow many chips would you like to bet ?  '))
            if self.bet == int(self.bet):
                print(f'Beting {self.bet} chips on the next hand')
                if self.bet > self.total:
                    print(f'Sorry, your bet cannot exceed {self.total}')
                    self.bet = int(input('\nHow many chips would you like to bet ?  '))
                    
        except ValueError:
            self.bet = int(input('Please try again'))
            
            
        
        
class Game:
    
    def __init__(self):
        ### connecting attributes for the game process ###
        self.player_chips = Chips()
        self.deck = Deck()
        self.player = Player(self.deck)



    def play_signle_game(self):
        
        ### where the magic happens ###
        
        playing = True
        
        while playing:
            
            self.gameover = False
            
            self.deck.shuffle()

            self.player_chips.take_bet()

            self.player_hand = Hand(self.deck)
            self.dealer_hand = Hand(self.deck, dealer=True)

            self.player_hand.hand = self.deck.deal_hand()
            self.dealer_hand.hand = self.deck.deal_hand()

            self.dealer_hand.display()
            self.player_hand.display()
            
            self.blackjack_check()
            
            while not self.gameover:

                choice = input("\nDo you want to [H]it, [S]tand -- or [R]ules: ").lower()

                ## player chose to hit ## 
                if choice.startswith('h'):
                    self.player_hand.draw_card()
                    self.player_hand.display()
                    self.blackjack_check()
                    self.bust_check()

                ## player chose to hit ## 
                elif choice.startswith('s'):
                    
                    
                    while self.dealer_hand.calculate_value() < 17:
                        self.dealer_hand.draw_card()
                        self.blackjack_check()
                        self.bust_check()
                    
                    player_hand_val = self.player_hand.calculate_value()
                    dealer_hand_val = self.dealer_hand.calculate_value()
                    
                    if self.gameover != True:
                        
                        ### will go in this loop if there is busts/blackjacks ###
                        
                        print(f'\nPlayer hand = {player_hand_val}')
                        print(f'Dealer hand = {dealer_hand_val}')
                        
                        ## player wins ##
                        if player_hand_val > dealer_hand_val:
                            self.total = (self.player_chips.win_bet() * 2)
                            print(f'\nYou won {self.player_chips.bet} chips, Total chips balace is : {self.player_chips.total}')
                            self.gameover = True
                            
                        ## player loses ##    
                        elif player_hand_val < dealer_hand_val:
                            self.total = self.player_chips.lose_bet()
                            print(f'\nYou lost {self.player_chips.bet} chips, Total chips balace is : {self.player_chips.total}')
                            self.gameover = True
                            
                        ## player loses ##    
                        else:
                            self.total = self.player_chips.lose_bet()
                            print(f'\nDraw!!! You lost {self.player_chips.bet} chips, Total chips balace is : {self.player_chips.total}')
                            self.gameover = True
                            
                ## print rules ## 
                elif choice.startswith('r'):
                    print(rules)
                    self.player_hand.display()

                else:
                    print('\nSomething went wrong, please try again')
                    self.gameover = False
                    
            ## after finishing playing a hand > ask to replay ##
            again = input("\nDo you want to play again? (Y/N) : ")
            if again.lower().startswith('y'):
                self.gameover = False
            elif again.lower().startswith('n'):
                print(f'\nThanks you for playing, total chips are : {self.player_chips.total}')
                playing = False
            else:
                print('\nPlease try again')
                again = input("\nDo you want to play again? (Y/N) : ")

                

    def blackjack_check(self):
        if self.dealer_hand.calculate_value() == 21:
            self.total = self.player_chips.lose_bet()
            self.dealer_hand.__str__()
            print(f'\n ~~~ Dealer have a blackjack, you lost {self.player_chips.bet} chips ~~~')
            print(f'\nTotal chip balance is : {self.player_chips.total}')
            self.gameover = True

        elif self.player_hand.calculate_value() == 21:
            self.total = (self.player_chips.win_bet() * 2)
            print(f'\n ~~~ You have a blackjack, You won {(self.player_chips.bet * 2)} chips ~~~')
            print(f'\nTotal chip balance is : {self.player_chips.total}')
            self.gameover = True
            
            
            
    def bust_check(self):   
        if self.dealer_hand.calculate_value() > 21:
            self.total = (self.player_chips.win_bet() * 2)
            self.dealer_hand.__str__()
            print(f'\nDealer bust, You won {self.player_chips.bet} chips, Total chips balace is : {self.player_chips.total}')
            self.gameover = True

        elif self.player_hand.calculate_value() > 21:
            self.total = self.player_chips.lose_bet()
            print(f'\nBust, You lost {self.player_chips.bet} chips, Total chips balace is : {self.player_chips.total}')
            self.gameover = True
            
            
            
            
            
    
print('\nHello, welcome to Blackjack!')
g = Game()
g.play_signle_game()



Hello, welcome to Blackjack!

How many chips would you like to bet ?  15
Beting 15 chips on the next hand

First card in the dealer's hand is hidden : X & J

Your cards are : ['A', '4'] with value of : 15

Do you want to [H]it, [S]tand -- or [R]ules: h

Your cards are : ['A', '4', 'J'] with value of : 15

Do you want to [H]it, [S]tand -- or [R]ules: h

Your cards are : ['A', '4', 'J', '2'] with value of : 17

Do you want to [H]it, [S]tand -- or [R]ules: s

Player hand = 17
Dealer hand = 20

You lost 15 chips, Total chips balace is : 985

Do you want to play again? (Y/N) : y

How many chips would you like to bet ?  200
Beting 200 chips on the next hand

First card in the dealer's hand is hidden : X & 2

Your cards are : ['8', 'Q'] with value of : 18

Do you want to [H]it, [S]tand -- or [R]ules: s

Player hand = 18
Dealer hand = 20

You lost 200 chips, Total chips balace is : 785

Do you want to play again? (Y/N) : y

How many chips would you like to bet ?  6000
Beting 6000 chips on the